# UNeT Arch

In [ ]:
import torch
import torch.nn as nn
from torch.nn import init
import logging
from torchvision import models

def double_conv(in_c, out_c):
    conv = nn.Sequential(
        nn.Conv2d(in_c, out_c, kernel_size=(3,2),padding=1 ,stride=1, bias=True),
        nn.BatchNorm2d(out_c),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_c, out_c, kernel_size=(3,2), padding=1 ,stride=1, bias=True),
        nn.BatchNorm2d(out_c),
        nn.ReLU(inplace=True))
    return conv

def double_conv1(in_c, out_c):
    conv = nn.Sequential(
        nn.Conv2d(in_c, out_c, kernel_size=(3,2),padding=1 ,stride=1, bias=True),
        nn.BatchNorm2d(out_c),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_c, out_c, kernel_size=(3,2),stride=1, bias=True),
        nn.BatchNorm2d(out_c),
        nn.ReLU(inplace=True))
    return conv

def double_conv2(in_c, out_c):
    conv = nn.Sequential(
        nn.Conv2d(in_c, out_c, kernel_size=(3,3),padding=1 ,stride=1, bias=True),
        nn.BatchNorm2d(out_c),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_c, out_c, kernel_size=(3,3), padding=1 ,stride=1, bias=True),
        nn.BatchNorm2d(out_c),
        nn.ReLU(inplace=True))
    return conv

def double_conv3(in_c, out_c):
    conv = nn.Sequential(
        nn.Conv2d(in_c, out_c, kernel_size=3,padding=1 ,stride=1, bias=True),
        nn.BatchNorm2d(out_c),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_c, out_c, kernel_size=3, padding=1 ,stride=1, bias=True),
        nn.BatchNorm2d(out_c),
        nn.ReLU(inplace=True))
    return conv

def double_conv4(in_c, out_c):
    conv = nn.Sequential(
        nn.Conv2d(in_c, out_c, kernel_size=(2, 3),padding=1 ,stride=1, bias=True),
        nn.BatchNorm2d(out_c),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_c, out_c, kernel_size=(2, 3), padding=1 ,stride=1, bias=True),
        nn.BatchNorm2d(out_c),
        nn.ReLU(inplace=True))
    return conv

def up_conv(in_c, out_c):
    conv = nn.Sequential(
        nn.ConvTranspose2d(in_c, out_c, kernel_size=2, stride=2))
    return conv

def up_conv1(in_c, out_c):
    conv = nn.Sequential(
        nn.ConvTranspose2d(in_c, out_c, kernel_size=(3,2),stride=2))
    return conv

def crop_ig(one, two):
    ts = two.size()[3]
    tens = one.size()[3]
    print(one.size()[3], two.size()[3])
    delta = abs(tens - ts)
    delta = delta
    return one[:, :, delta: tens-delta, delta: tens-delta]

class UNet(nn.Module):
    def __init__(self,img_ch=2,output_ch=2):
        super(UNet, self).__init__()

        self.max_pool_2x2 = nn.MaxPool2d(kernel_size=(2,1), stride=2)
        self.down_conv_1 = double_conv1(img_ch, 64)
        self.down_conv_2 = double_conv2(64, 128)
        self.down_conv_3 = double_conv3(128, 256)

        self.up_trans_3 = up_conv1(256, 128)
        self.up_conv_3 = double_conv2(256, 128)
        
        self.up_trans_4 = up_conv(128, 64)
        self.up_conv_4 = double_conv4(128, 64)
        
        self.out = nn.Conv2d(
            in_channels=64,
            out_channels=output_ch,
            kernel_size=1,stride=1,padding=0)
    
    def forward(self, image):
        # encoder
        x1 = self.down_conv_1(image)
        x2 = self.max_pool_2x2(x1)
        x3 = self.down_conv_2(x2)
        x4 = self.max_pool_2x2(x3)
        x5 = self.down_conv_3(x4)
        
        # decoder
        x = self.up_trans_3(x5)
        x = self.up_conv_3(torch.cat([x, x3], 1))
        x = self.up_trans_4(x)
        x = self.up_conv_4(torch.cat([x, x1], 1))
        # output
        x = self.out(x)
    
        return x



In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# torch.manual_seed(17)

# if torch.cuda.is_available():
#     device = torch.device('cuda:0')
#     torch.backends.cudnn.deterministic = True
#     torch.backends.cudnn.benchmark = False
# else:
#     device = torch.device('cpu')

# print(device)

In [ ]:
import scipy.io as sio
import numpy as np
import torch
from torch import nn, optim
import torchvision
from torch.utils.data import Dataset, DataLoader
import numpy as np 
import math
import pandas as pd
import cmath
from torch.utils.data.sampler import SubsetRandomSampler

In [ ]:
 from torch.autograd import Variable

In [ ]:

df1 = sio.loadmat("./gdrive/MyDrive/DOA/preTrain/SNR_NS_ALL_50000_1.mat")

In [ ]:

max_r = 0.2655554840881041
max_i = 0.247665118111416
max_p = 3.141592030578154
min_r = -0.24698739374092077
min_i = -0.23956457294409056
min_p = -3.1415875527493355

class DOA_dataset(Dataset):
    def __init__(self, df):
        transp = np.transpose(df['NS_data'], (2, 0, 1))
        new = np.zeros((300000, 3, 8, 100))
        for i in range(0, transp.shape[0]):
            for j in range(0, transp.shape[1]):
                for k in range(0, transp.shape[2]):
                    new[i][0][j][k] = (transp[i][j][k].real - min_r)/(max_r-min_r)
                    new[i][1][j][k] = (transp[i][j][k].imag - min_i)/(max_i-min_i)
                    new[i][2][j][k] = (cmath.phase(transp[i][j][k]) - min_p)/(max_p-min_p)

        self.x = torch.from_numpy(new)
        self.y = torch.from_numpy(np.asarray(df['DOA']))
        self.n_sample = len(self.y)
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    def __len__(self):
        return self.n_sample

In [ ]:
dataset = DOA_dataset(df1)

validation_split = .3
shuffle_dataset = True
random_seed= 42

# Creating data indices for training and validation splits:
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)


#dataloader = DataLoader(dataset=dff, batch_size=100, shuffle=True,  num_workers=2)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=128, 
                                           sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(dataset, batch_size=256,
                                                sampler=valid_sampler)


In [ ]:
unet = UNet()

optimizer = optim.Adam(unet.parameters(), lr=0.001)

criterion = nn.BCELoss ()

# Using GPU if available 
if torch.cuda.is_available():
  print(torch.cuda.get_device_name(0))
  unet = unet.cuda()
  criterion = criterion.cuda()

# exp_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=40, gamma = 0.1)
def train():
  ''' Funtion for training  '''

  for i in range(200):
    # Dataset Training Loss
    training_loss = 0
    for features, labels in train_loader:
      inputs, labels = Variable(features.cuda()), Variable(labels.cuda())
      optimizer.zero_grad()
      output = unet(inputs.float().cuda())
      output = torch.sigmoid(output)
      losss = criterion(output, inputs.float().cuda())
      losss.backward()
      optimizer.step()
      
      # exp_scheduler.step()
      training_loss += losss.item()

    # Dataset Validation
    validation_loss = 0
    with torch.no_grad():
      for features, labels in validation_loader:
        inputs, labels = Variable(features.cuda()), Variable(labels.cuda())
        output = unet(inputs.float().cuda())
        output = torch.sigmoid(output)
        loss = criterion(output, inputs.float().cuda())
        validation_loss += loss.item()
    print("Epoch {} - Traningloss: {}".format(i+1, training_loss/len(train_loader)))
    print("Validationloss: {}".format( validation_loss/len(validation_loader)))

  # Saving Model as .pth file
  torch.save(unet.state_dict(), "./Attention_block_wieghts_phase.pth")
  print("Training complete, model weights are saved")

Tesla T4


In [ ]:
train()

Epoch 1 - Traningloss: 0.3005560276183215
Validationloss: 0.14635395569105944
Epoch 2 - Traningloss: 0.11316394778815182
Validationloss: 0.08797298185527325
Epoch 3 - Traningloss: 0.08479482010006904
Validationloss: 0.0744174225255847
Epoch 4 - Traningloss: 0.07720596268773079
Validationloss: 0.06982004782184958
Epoch 5 - Traningloss: 0.07456880936568433
Validationloss: 0.06806276862819989
Epoch 6 - Traningloss: 0.07302839715372432
Validationloss: 0.06709852674975991
Epoch 7 - Traningloss: 0.07226866846057502
Validationloss: 0.06592612372090419
Epoch 8 - Traningloss: 0.07180313833735207
Validationloss: 0.0657788875202338
Epoch 9 - Traningloss: 0.0712685451419516
Validationloss: 0.06553743702049057
Epoch 10 - Traningloss: 0.07115896230732853
Validationloss: 0.06587582019468148
Epoch 11 - Traningloss: 0.07119608938016675
Validationloss: 0.06556447595357895
Epoch 12 - Traningloss: 0.07089409855279055
Validationloss: 0.0877057119893531
Epoch 13 - Traningloss: 0.0721733203327114
Validationl

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 8))
plt.plot(epc_list, train_loss_list, label='Train')
plt.plot(epc_list, val_loss_list, label='Valid')
plt.xlabel('Epochs', fontsize=14)
plt.ylabel('Loss', fontsize=14)
plt.legend(fontsize=14)
plt.show()